<a href="https://colab.research.google.com/github/pedropalmav/proyecto-recsys/blob/rlhf/RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-11-29 15:43:28--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc791bd87a4edc7b0edbd31cc691.dl.dropboxusercontent.com/cd/0/inline/CfRvvVMqEjcBxtpyAORG6BlkDys_96VldJQpfBJnWzUSLNyOcrJ9Byaz_UtLTtRzqJm1KpTG_0Kq2WIw3mIkQklC9ZerIrxM6WPa9_ISidYvaS3LYwmHBKopoJOhgHvXPpY/file?dl=1# [following]
--2024-11-29 15:43:29--  https://uc791bd87a4edc7b0edbd31cc691.dl.dropboxusercontent.com/cd/0/inline/CfRvvVMqEjcBxtpyAORG6BlkDys_96VldJQpfBJnWzUSLNyOcrJ9Byaz_UtLTtRzqJm1KpTG_0Kq2WIw3mIkQklC9ZerIrxM6WPa9_ISidYvaS3LYwmHBKopoJOhgHvXPpY/file?dl=1
Resolving uc791bd87a4edc7b0edbd31cc691.dl.dropboxusercontent.com (uc791bd87a4edc7b0edbd31cc691.dl.dropboxusercontent.com)... 162.125.65.15, 2620:100:6021:15::

## Load HF

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft
!pip install datasets
!pip install trl
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 11.4 MB/s eta 0:00:00

# Supervised Fine Tuning (SFT)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, pipeline
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = f"Pretend you are a movie recommender system. I will give you a conversation between a human and assistant. Based on the conversation, you reply me with 10 recommendations without extra sentences. Here is the conversation: {conversation}."
    chat = [
        {"role": "user", "content": content}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

In [ ]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [ ]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [ ]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

In [ ]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.01 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [ ]:
prompts = []
ground_truth = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  conversation_id = user_data['Conversation'][0]['conversation_1']['conversation_id']
  prompt = generate_zeroshot(conversation_id, conversations)
  prompts.append(prompt)


In [ ]:
from datasets import Dataset

dataset = Dataset.from_list(conversations)

def flatten_and_tokenize_function(examples):
    combined_texts = []
    for messages in examples['messages']:
        flattened_messages = []
        for message in messages:
            if isinstance(message, list):
                flattened_messages.extend(message)
            else:
                flattened_messages.append(message)
        combined_texts.append(" ".join(flattened_messages))

    return tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(flatten_and_tokenize_function, batched=True, remove_columns=["conversation_id", "messages"])

Map:   0%|          | 0/10089 [00:00<?, ? examples/s]

In [26]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=256,
    peft_config=peft_config
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:160: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the o

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 99.06 MiB is free. Process 22909 has 14.65 GiB memory in use. Of the allocated memory 14.33 GiB is allocated by PyTorch, and 204.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)